# Vegetable Prices

In [ ]:
import requests
import json
from bs4 import BeautifulSoup

In [ ]:
url = "http://www.statistics.gov.lk/DashBoard/Prices/Prices_Data.php"

# Send a GET request to the URL
response = requests.get(url)

In [ ]:
name_list = ["productList.json", "pricesList.json", "unavailability.json"]

In [ ]:
if response.status_code == 200:
    # Extract the data from the response (assuming it's in JSON format)
    response_text = response.content.decode('utf-8-sig')

    response_text_filtered = response_text.split("// ")
    modified_data = {}
    for i in range(len(response_text_filtered)):
        if(i == 0):
            continue
        modified_data[i] = '\n'.join(line for line in response_text_filtered[i].split('\n') if not line.startswith("Data for"))
        modified_data[i] = '\n'.join(line for line in modified_data[i].split('\n') if not line.startswith("var"))
        modified_data[i] = '\n'.join(line for line in modified_data[i].split('\n') if not line.startswith("]"))
        # wrap modified data with square brackets
        modified_data[i] = "[" + modified_data[i] + "]"
        # remove empty values
        modified_data[i] = modified_data[i].replace("''", "null") 
        with open('./data/unprocessed/'+name_list[i-1], 'w') as outfile:
            outfile.write(modified_data[i])

# Fuel Prices

In [ ]:
response = requests.get("https://ceypetco.gov.lk/wp-json/wp/v2/pages/248")

In [ ]:
data = response.json()
data = data['content']['rendered']
soup = BeautifulSoup(data, 'html.parser')
target_table = soup.find('table', class_="ea-advanced-data-table ea-advanced-data-table-static ea-advanced-data-table-9fdc03b ea-advanced-data-table-sortable ea-advanced-data-table-paginated ea-advanced-data-table-searchable")
price_list = []
rows = target_table.find_all('tr')
headers = [header.get_text().strip() for header in rows[0].find_all('th')]

In [ ]:
for row in rows[1:]:
    cells = row.find_all('td')
    if cells:
        price_data = {"Date": cells[0].get_text().strip()}
        for i, cell in enumerate(cells[1:], 1):
            value = cell.get_text().strip()
            parts = value.split(".")
            if len(parts) > 2:
                value = parts[0] + '.' + ''.join(parts[1:])
            value.replace(',', '')
            try:
                price_data[headers[i]] = float(value)
            except ValueError:
                price_data[headers[i]] = None
        price_list.append(price_data)

In [ ]:
with open('./data/unprocessed/ceypetcoPrices.json', 'w') as f:
    json.dump(price_list, f, indent=4)